### Data Visualizations of 2022 Business Data - County Maps 

Goal:

A Jupyter notebook that produces a map dividing an area of interest into regions according to how many grocery stores the population has access to.

In [ ]:
import utils.simulation as sim 
import utils.marketmaker as mm 
import pandas as pd
import geopandas as gpd 
import folium
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [ ]:
#reading in dataset that uses marketmaker function that cleans the dataset and locates just the grocery stores
grocery22 = mm.supermarket_generator('/srv/data/my_shared_data_folder/rafi/2022_Business_Academic_QCQ_grocery.csv','costco_2022.csv','2022_Business_Academic_QCQ_supercenters.csv')
supermarket22 = mm.group(grocery22)

# Los Angeles, CA Area Analysis 

### WEST COAST

In [ ]:
#read in the census data file 
census_counties = gpd.read_file('../../rafi-grocery/Data/County Boundaries /cb_2018_us_county_20m.shp')
census_counties.head()

In [ ]:
#removing US territories 
territories = ['72', '69', '66', '78','60','02','15']
updated_census_counties = census_counties.drop(census_counties[census_counties['STATEFP'].isin(territories)].index)
updated_census_counties.head()

updated_census_counties['COUNTYFP'] = updated_census_counties['COUNTYFP'].astype(float)
updated_census_counties.head()

In [ ]:
#States with their numbers and abbreviations 
num_abb =  {
    '01': 'AL',
    '02': 'AK',
    '04': 'AZ',
    '05': 'AR',
    '06': 'CA',
    '08': 'CO',
    '09': 'CT',
    '10': 'DE',
    '12': 'FL',
    '13': 'GA',
    '15': 'HI',
    '16': 'ID',
    '17': 'IL',
    '18': 'IN',
    '19': 'IA',
    '20': 'KS',
    '21': 'KY',
    '22': 'LA',
    '23': 'ME',
    '24': 'MD',
    '25': 'MA',
    '26': 'MI',
    '27': 'MN',
    '28': 'MS',
    '29': 'MO',
    '30': 'MT',
    '31': 'NE',
    '32': 'NV',
    '33': 'NH',
    '34': 'NJ',
    '35': 'NM',
    '36': 'NY',
    '37': 'NC',
    '38': 'ND',
    '39': 'OH',
    '40': 'OK',
    '41': 'OR',
    '42': 'PA',
    '44': 'RI',
    '45': 'SC',
    '46': 'SD',
    '47': 'TN',
    '48': 'TX',
    '49': 'UT',
    '50': 'VT',
    '51': 'VA',
    '53': 'WA',
    '54': 'WV',
    '55': 'WI',
    '56': 'WY',
    '11': 'DC',
    }


In [ ]:
#replaces the STATEFP column values with state abbreviations from our data
updated_census_counties['STATEFP'] = updated_census_counties['STATEFP'].replace(num_abb)
updated_census_counties.head()

In [ ]:
#merging census data with our supermarkets data
#first line of code renames the STATEFP and COUNTYFP columns to match our data 
census_counties_2 = updated_census_counties.rename(columns={'STATEFP': 'STATE','COUNTYFP': 'COUNTY CODE'})
censusboundaries= pd.merge(census_counties_2, supermarkets, on=['STATE', 'COUNTY CODE'], how='left')
censusboundaries

In [ ]:
#filtering dataframe to just california 
cali_data =censusboundaries[censusboundaries['STATE'] == 'CA']
cali_data.plot()

In [ ]:
cali_data.NAME.unique()

In [ ]:
#picking the desrired counties for the LA area - Los Angeles, Orange, Riverside, San Bernadino, Ventura
desired_counties = ['Los Angeles', 'Orange', 'Riverside','San Bernardino', 'Ventura']  

#Filtering original dataframe by these desired counties for just LA and plotting
la_data = cali_data[cali_data['NAME'].isin(desired_counties)]
la_data.plot()



In [ ]:
#plotting the above map using plotly 
fig, ax = plt.subplots(figsize=(10, 10))
la_data.plot(ax=ax, column='NAME', cmap='Blues', edgecolor='gray', linewidth=0.8)
ax.set_title('Los Angeles desired counties')
ax.set_axis_off()
plt.show()

In [ ]:
## Kroger Stores only  in LA 

kro_la = la_data.query("`STORE TYPE` == 'KROGER'")

In [ ]:
## Albertson Stores only in LA 

alb_la = la_data.query("`STORE TYPE` == 'ALBERTSONS'")

# Proximity Analysis 

### Data Visualizations of 2022 Business Data - County Maps 


Goal:

A Jupyter notebook that produces a proximity analysis of Kroger-albertsons stores after the merger for a specific region

In [ ]:
import folium
from folium import Marker, GeoJson
from folium.plugins import HeatMap

import pandas as pd
import geopandas as gpd

In [ ]:
#making sure the geodataframe using the same coordinate reference system(crs)
#however crs is not suitable for measuring distance 
print(kro_la.crs)
print(alb_la.crs)

In [ ]:
kro_la=kro_la.set_crs(2229,allow_override=True)
alb_la=alb_la.set_crs(2229,allow_override=True)

In [ ]:
kro_la.crs

In [ ]:
#Creating a GeoSeries two_half_mile_buffer (2.5*5280 feet) 

distance=2.5*5280
two_half_mile_buffer = kro_la.geometry.buffer(distance)
two_half_mile_buffer.plot()

# Albertsons-Kroger Stores Locations - Los Angeles, CA

In [ ]:
# Create map with kroger and albertson stores 

m = folium.Map(location=[34.052235,-118.243683], zoom_start=11)


for idx, row in kro_la.iterrows():
    Marker([row['LATITUDE'], row['LONGITUDE']],popup='Kroger',icon=folium.Icon(color='darkblue',icon='K')).add_to(m)

for idx, row in alb_la.iterrows():
    Marker([row['LATITUDE'], row['LONGITUDE']],popup='Albertsons',icon=folium.Icon(color='darkred',icon='A')).add_to(m)
    

# Show the map
m

# 2.5 Mile Radius Analysis 

In [ ]:
# Create map with kroger and albertson stores with 2.5 mile radius circles applied 

m = folium.Map(location=[34.052235,-118.243683], zoom_start=11)


for idx, row in kro_la.iterrows():
    folium.Marker([row['LATITUDE'], row['LONGITUDE']],popup='Kroger',icon=folium.Icon(color='darkblue',icon='K')).add_to(m)
    folium.Circle(location=[row['LATITUDE'], row['LONGITUDE']],fill_color="#008080",radius=distance,weight=1,color="#008080").add_to(m)

for idx, row in alb_la.iterrows():
    folium.Marker([row['LATITUDE'], row['LONGITUDE']],popup='Albertsons',icon=folium.Icon(color='red',icon='A')).add_to(m)
    
# Plot each polygon on the map
#folium.GeoJson(two_half_mile_buffer.to_crs(epsg=2229)).add_to(m)

# Show the map
m

From the map above, a 2.5 mile radius was used to examine proximity and we can see that many Kroger-Albertsons stores fall within this boundary.